In [20]:
import pandas as pd 
import yfinance as yf  
import numpy as np
import datetime as dt 
from datetime import datetime,date
from dateutil.relativedelta import relativedelta

In [21]:
ticker='^GDAXI'
sd=date.today()-relativedelta(days=729)
ed=date.today()


In [22]:
df=yf.download(tickers=ticker,start=sd,end=ed,interval='1h')


[*********************100%***********************]  1 of 1 completed


In [23]:
df

,Open,High,Low,Close,Adj Close,Volume
2020-03-17 09:00:00+01:00,9141.169922,9143.639648,8610.589844,8672.419922,8672.419922,0
2020-03-17 10:00:00+01:00,8913.790039,8929.559570,8423.559570,8519.269531,8519.269531,0
2020-03-17 11:00:00+01:00,8563.730469,8782.360352,8423.559570,8780.940430,8780.940430,0
2020-03-17 12:00:00+01:00,8730.599609,8782.360352,8532.929688,8564.480469,8564.480469,0
2020-03-17 13:00:00+01:00,8616.230469,8655.980469,8532.929688,8649.910156,8649.910156,0
...,...,...,...,...,...,...
2022-03-15 13:00:00+01:00,13762.769531,13873.330078,13756.379883,13845.360352,13845.360352,0
2022-03-15 14:00:00+01:00,13844.410156,13896.019531,13777.709961,13855.759766,13855.759766,0
2022-03-15 15:00:00+01:00,13855.650391,13930.450195,13791.019531,13906.469727,13906.469727,0
2022-03-15 16:00:00+01:00,13906.360352,13974.839844,13812.500000,13843.719727,13843.719727,0


In [24]:
df['prior_close']=df['Close'].shift(+1)

df['date_index']=df.index

df['time']=df.index
df['time']=df['time'].dt.time
df['time']=df['time'].astype('str')

df['date']=df.index
df['date']=df['date'].dt.date

df['gap'] = df['Open']/df['prior_close']-1

In [25]:
prior_close=df[df['time']=='09:00:00']

In [26]:
prior_close=prior_close[['date','prior_close','gap']]
prior_close['date']=prior_close['date'].astype('str')
df['date']=df['date'].astype('str')
unique_date=prior_close['date'].unique()
unique_date=unique_date.tolist()

In [27]:
filter_df=pd.DataFrame()
tf_df=pd.DataFrame()
tf_list=[]
for i in unique_date:
    cl=prior_close.loc[prior_close['date']==i,'prior_close'] [0]
    tf=df[df['date']==i]
    tf['last_day_close']=cl

    return_l=prior_close.loc[prior_close['date']==i,'gap'][0]
    rf=df[df['date']==i]
    rf['opening_gap']=return_l
    tf['opening_gap']=rf['opening_gap']


    gapup_tf=tf[tf['opening_gap']>0.005]
    gapup_tf['close_gap_y/n']=np.where(gapup_tf['Low']<=gapup_tf['last_day_close'],1,0)

    gapdown_tf=tf[tf['opening_gap']<-0.005]
    gapdown_tf['close_gap_y/n']=np.where(gapdown_tf['High']>=gapdown_tf['last_day_close'],1,0)

    tf=pd.concat([gapup_tf,gapdown_tf])

    total=tf['close_gap_y/n'].sum()
    tf['close_within_day']=total

    filter_2=tf[tf['close_within_day']<1]
    filter_2=filter_2[filter_2['close_gap_y/n']==0]
    try:
        filter_2=filter_2.iloc[0]
    except:
        filter_2=filter_2[filter_2['close_gap_y/n']==0]

    filter_1=tf[tf['close_gap_y/n']==1]
    try:
        filter_1=filter_1.iloc[0]
    except:
            filter_1=tf[tf['close_gap_y/n']==1]
    filter_df=filter_df.append(filter_1,ignore_index=True)
    filter_df=filter_df.append(filter_2,ignore_index=True)



df=filter_df
df=df.set_index('date_index')
df=df[['High','Low','Open','last_day_close','opening_gap','close_gap_y/n','time','date']]

/Users/anderslindstrom/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Users/anderslindstrom/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/Users/anderslindstrom/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a 

In [28]:
conditions=[
    (df['close_gap_y/n'] ==1)&(df['time']=='09:00:00'),
    (df['close_gap_y/n'] ==1)&(df['time']=='10:00:00'),
    (df['close_gap_y/n'] ==1)&(df['time']=='11:00:00'),
    (df['close_gap_y/n'] ==1)&(df['time']=='12:00:00'),
    (df['close_gap_y/n'] ==1)&(df['time']=='13:00:00'),
    (df['close_gap_y/n'] ==1)&(df['time']=='14:00:00'),
    (df['close_gap_y/n'] ==1)&(df['time']=='15:00:00'),
    (df['close_gap_y/n'] ==1)&(df['time']=='16:00:00'),
    (df['close_gap_y/n'] ==1)&(df['time']=='17:00:00'),
    (df['close_gap_y/n'] ==0)
    ]
values=['1h_close','2h_close','3h_close','4h_close','5h_close','6h_close','7h_close','8h_close','9h_close','no_close']

conditions2=[
    #gap_up
    (df['opening_gap'] >=0.005)&(df['opening_gap']<0.01),
    (df['opening_gap'] >=0.01)&(df['opening_gap']<0.015),
    (df['opening_gap'] >=0.015)&(df['opening_gap']<0.02),
    (df['opening_gap'] >=0.02),
    #gap_down
    (df['opening_gap'] <=-0.005)&(df['opening_gap']>-0.01),
    (df['opening_gap'] <=-0.01)&(df['opening_gap']>-0.015),
    (df['opening_gap'] <=-0.015)&(df['opening_gap']>-0.02),
    (df['opening_gap'] <=0.02),
    ]
values2=['0.5/1%','1/1.5%','1.5/2%','>2%','-0.5/-1%','-1/-1.5%','-1.5/-2%','<-2%']

In [29]:
df['closing_time']=np.select(conditions,values)
df['gap_cat']=np.select(conditions2,values2)


In [30]:
cross_tab=pd.crosstab(df['gap_cat'],df['closing_time'])
cross_tab=pd.DataFrame(cross_tab)

In [31]:
cross_tab['sum']=cross_tab.sum(axis=1)
cross_tab['1h_close']=cross_tab['1h_close']/cross_tab['sum']
cross_tab['2h_close']=(cross_tab['2h_close']/cross_tab['sum'])+cross_tab['1h_close']
cross_tab['3h_close']=(cross_tab['3h_close']/cross_tab['sum'])+cross_tab['2h_close']
cross_tab['4h_close']=(cross_tab['4h_close']/cross_tab['sum'])+cross_tab['3h_close']
cross_tab['5h_close']=(cross_tab['5h_close']/cross_tab['sum'])+cross_tab['4h_close']
cross_tab['6h_close']=(cross_tab['6h_close']/cross_tab['sum'])+cross_tab['5h_close']
cross_tab['7h_close']=(cross_tab['7h_close']/cross_tab['sum'])+cross_tab['6h_close']
cross_tab['8h_close']=(cross_tab['8h_close']/cross_tab['sum'])+cross_tab['7h_close']
cross_tab['9h_close']=(cross_tab['9h_close']/cross_tab['sum'])+cross_tab['8h_close']
cross_tab['no_close']=(cross_tab['no_close']/cross_tab['sum'])

cross_tab=cross_tab.reindex(['<-2%','-1.5/-2%','-1/-1.5%','-0.5/-1%','0.5/1%','1/1.5%','1.5/2%','>2%'])


In [32]:
cross_tab

closing_time,1h_close,2h_close,3h_close,4h_close,5h_close,6h_close,7h_close,8h_close,9h_close,no_close,sum
gap_cat,,,,,,,,,,,
<-2%,0.000000,0.000000,0.066667,0.066667,0.133333,0.200000,0.266667,0.333333,0.333333,0.666667,15
-1.5/-2%,0.000000,0.090909,0.090909,0.090909,0.181818,0.181818,0.181818,0.181818,0.181818,0.818182,11
-1/-1.5%,0.058824,0.088235,0.088235,0.147059,0.176471,0.205882,0.264706,0.323529,0.352941,0.647059,34
-0.5/-1%,0.125000,0.275000,0.350000,0.350000,0.350000,0.400000,0.425000,0.425000,0.425000,0.575000,40
0.5/1%,0.103896,0.142857,0.194805,0.220779,0.220779,0.285714,0.350649,0.402597,0.402597,0.597403,77
1/1.5%,0.055556,0.111111,0.138889,0.166667,0.166667,0.166667,0.194444,0.250000,0.277778,0.722222,36
1.5/2%,0.000000,0.000000,0.000000,0.000000,0.142857,0.285714,0.285714,0.285714,0.285714,0.714286,7
>2%,0.000000,0.000000,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.916667,12
